In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
import os
from pathlib import Path

path_cwd=Path.cwd()
path_input=str(path_cwd)+'/d_input/'
#path_output=str(path_cwd)+'/Data_output/'


In [8]:
#read
df = pd.read_csv(path_input+'ENVIRONMENTAL_tidy.csv', index_col='key_0',parse_dates=['key_0'])
sf_df=pd.read_csv(path_input+'SF_27.csv', index_col= 'Unnamed: 0',parse_dates=['Unnamed: 0'])

In [9]:
# In-Out
Precipitation_2021= df.filter(items=['Rain(mm)']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
PET_2021= df.filter(items=['PET(mm/h)']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
Precipitation_2022= df.filter(items=['Rain(mm)']).loc['2022-05-01 00:30:00':'2022-09-15 00:30:00']
PET_2022= df.filter(items=['PET(mm/h)']).loc['2022-05-01 00:30:00':'2022-09-15 00:30:00']

#SF
SF_2021=sf_df.filter(items=['Total SF_LR (cm3/h)']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
sf_orig = ((SF_2021['Total SF_LR (cm3/h)'].values)*0.001).tolist()

#SM
SM_2021=sf_df.filter(items=['S_avg']).loc['2021-05-01 00:30:00':'2021-09-15 00:30:00']
sm_orig = (SM_2021['S_avg'].values).tolist() # 5 cm3 of contributing area, 0.001 cm3 -> dm3

#### Parameters (capacities and flow)
- X: concentrated fraction that is lost from P to seepage almost immediately 
- sf_max: maximum capacity for surface fracture L
- df_max: maximum capacity for deep fracture L
- tree_max: maximum capacity for tree L

In [7]:
#input data 
precipitation=((Precipitation_2021['Rain(mm)'].values)*50856*1e-6).tolist() #50,856 mm2 area of rain gauge. 1e-6 mm3 to dm3 

pet=PET_2021['PET(mm/h)'].tolist() #calculated every 30 min and assumed to be over 1m2 -> mm~L

dates=Precipitation_2021.index.tolist()

In [8]:
from sfmodel1 import sfmodel

m = sfmodel(shape=len(Precipitation_2021),params={'X': 0.03, 'sf_max': 1, 'df_max': 3, 'tree_max': 2},data={'precipitation': precipitation, 'pet': pet, 'dates': dates})

# Reservoirs
m.surface_fracture = [None]*m.shape
m.deep_fracture = [None]*m.shape
m.quickflow= [None]*m.shape
m.tree = [None]*m.shape

#Initial stocks
m.surface_fracture[0] = 0.1 # L
m.deep_fracture[0] = 0.3 # L
m.quickflow[0] = 0.001 # L
m.tree[0] = 0.1 #L

# Flows- Record all flow channels
m.q1 = [None]*m.shape # precipitation -> quickflow
m.q2 = [None]*m.shape # precipitation -> soil
m.q3 = [None]*m.shape # soil -> fracture
m.q3_1 = [None]*m.shape # fracture -> soil
m.q3_2 = [None]*m.shape #soil -> quickflow
m.q4 = [None]*m.shape # fracture -> plant
m.q4_1 = [None]*m.shape # fracture -> air
m.q5 = [None]*m.shape # soil -> plant
m.q5_1 = [None]*m.shape # soil -> air
m.q6 = [None]*m.shape # plant -> air
m.sapflow = [None]*m.shape # plant -> air

In [ ]:
precipitation = m.data['precipitation']